In [2]:
import datasets
import os
import pandas as pd

import transformers
import torch
import tensorflow as tf;
import re
from pathlib import Path
import subprocess
import json
from tqdm.notebook import tqdm
import time

# import multiprocess
# multiprocess.set_start_method("spawn")


REPOS_DIR = os.getcwd()+'/data/repos'

gvm_root = os.environ['GVM_ROOT']
os.environ['PATH'] = f"{gvm_root}/bin:{gvm_root}/pkgsets/go1.24.2/global/bin:{gvm_root}/gos/go1.24.2/bin:{gvm_root}/pkgsets/go1.24.2/global/overlay/bin:{os.environ['PATH']}"
goimports_binary = f"{gvm_root}/pkgsets/go1.24.2/global/bin/goimports"

2025-05-02 12:29:09.572900: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746178149.676067 3448335 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746178149.705366 3448335 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746178149.911712 3448335 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746178149.911771 3448335 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746178149.911777 3448335 computation_placer.cc:177] computation placer alr

In [2]:
# !pip install --upgrade pip
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126
# !pip install transformers
# !pip install tensorflow[and-cuda]
# !pip install tf-keras
# !pip install 'accelerate>=0.26.0'
# !pip install bitsandbytes

In [3]:
import torch
x = torch.rand(5, 3)
print(x)

print(transformers.pipeline('sentiment-analysis')('hugging face is the best'))

print(tf.config.list_physical_devices('GPU'))

tensor([[0.5177, 0.8500, 0.3364],
        [0.4232, 0.2740, 0.1350],
        [0.6377, 0.6896, 0.5745],
        [0.9409, 0.9548, 0.1295],
        [0.3999, 0.3313, 0.5006]])


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0


[{'label': 'POSITIVE', 'score': 0.999839186668396}]
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
ds = datasets.Dataset.load_from_disk('data/test_candidates_ds')

print(len(ds))

ds[0]

3604


{'path': '/main.go',
 'repo_name': 'avelino/awesome-go',
 'revision_id': 'c3643eb9da5c673101f8fe15a6deb40bfc4a1c85',
 'star_events_count': 112752,
 'fork_events_count': 13739,
 'is_vendor': False,
 'is_generated': False,
 'length_bytes': 8755,
 'extension': 'go',
 'filename': 'main.go',
 'file_exists': True,
 'file_hash': '105132c75e5c98fa0379971e18dd1bb6544cabc7c5b18a81ae44585a4db25705',
 'project': '/',
 'relative_project_path': 'main.go',
 'relative_go_package': '',
 'funcs': 'main,buildStaticSite,dropCreateDir,mkdirAll,renderCategories,renderSitemap,extractCategories,extractCategory,rewriteLinksInIndex,renderIndex',
 'methods': '',
 'project_path': 'c3643eb9da5c673101f8fe15a6deb40bfc4a1c85/'}

In [3]:
import gc

try:
    del tokenizer
    del model
except Exception:
    pass
#model.cpu()
#del tokenizer, model

gc.collect()
torch.cuda.empty_cache()

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [4]:
gc.collect()
torch.cuda.empty_cache()

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [5]:
# model_name = 'deepseek-ai/deepseek-coder-6.7b-base'
# model_name = 'deepseek-ai/deepseek-coder-6.7b-instruct'
# model_name = 'deepseek-ai/deepseek-coder-1.3b-base'
model_name = 'deepseek-ai/deepseek-coder-1.3b-instruct'

# tokenizer = transformers.AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-6.7b-instruct", trust_remote_code=True)
# model = transformers.AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-6.7b-instruct", trust_remote_code=True, torch_dtype=torch.bfloat16).cuda()

tokenizer = transformers.AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = transformers.AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, torch_dtype=torch.bfloat16,
    #quantization_config=transformers.BitsAndBytesConfig(load_in_4bit=True),
).cuda()

In [6]:
system_message = """
You are an expert programmer. 
You should only return output test file containing working code.
The user is going to give you code and would like to have unit tests for the first file.
All the other files are just dependencies to give you context of all the possible test cases to produce.
Cover all possible inputs and their respective outputs using tests.
Each subtest must be wrapped into t.Run
"""

def generate_unit_test(file_contents: list[str]) -> list[str]:
    chats = [[
        {"role": "system", "content": system_message},
        {"role": "user", "content": f"{file_content}"}
    ] for file_content in file_contents]

    # https://huggingface.co/docs/transformers/main/en/internal/tokenization_utils#transformers.PreTrainedTokenizerBase.apply_chat_template

    texts = [tokenizer.apply_chat_template(
        chat,
        tokenize=False,
        add_generation_prompt=True
    ) for chat in chats]

    model_inputs = tokenizer(texts, padding=True, return_tensors="pt", truncation=True, max_length=16000).to(model.device)

    # tokenizer.eos_token_id is the id of <|EOT|> token
    model.generation_config.pad_token_id = tokenizer.pad_token_id
    # with torch.no_grad()
    outputs = model.generate(**model_inputs, max_new_tokens=3096, do_sample=False, top_k=50,
        # top_p=0.95,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,eos_token_id=tokenizer.eos_token_id)


    outputs = [output[model_inputs['input_ids'].shape[1]:] for i, output in enumerate(outputs)]
    answers: str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    for i in range(len(answers)):
        answer = answers[i]
        go_content_start = answer.find('```go')+5
        go_content_end = answer.find('```', go_content_start)
        answer = answer[go_content_start:go_content_end]
        answers[i] = answer

    return answers


In [7]:
def check_result(row, test_file_content):
    REPOS_DIR = os.getcwd()+'/data/repos'
    project_path = REPOS_DIR+'/'+row['project_path']
    test_relative_project_path = row['relative_project_path'][:len(row['relative_project_path'])-3]+'_llm_test.go'
    test_file_path = project_path+test_relative_project_path
    cover_file_path = project_path+row['relative_project_path'][:len(row['relative_project_path'])-3]+'_llm_test_cover.out'

    for p in Path(project_path).glob("**/*_test.go"):
        p.unlink()

    with open(test_file_path, "w") as f:
        f.write(test_file_content)

    test_fns: list[str] = re.findall(r'func\W+(Test\w+)\(t\W+\*testing\.T\)\W*\{', test_file_content)

    p = subprocess.run([goimports_binary, "-w", test_file_path], cwd=project_path,
                        stdout=subprocess.PIPE,
                        stderr=subprocess.PIPE,
    )
    if p.returncode != 0:
        raise Exception(f"goimports: code {p.returncode}, 'strderr' {p.stderr}")

    # print('# main_file_content:', main_file_content, '# test_file_content', test_file_content, '# test_fns', test_fns, sep="\n\n")

    p = subprocess.run(["go", "mod", "tidy"], cwd=project_path,
                        stdout=subprocess.PIPE, 
                        stderr=subprocess.PIPE,
    )
    if p.returncode != 0:
        raise Exception(f"go mod tidy: code {p.returncode}, 'strderr' {p.stderr}")
        # print('go mod tidy: code', p.returncode, 'strderr', p.stderr)

    test_stderr = ''
    p = subprocess.run(["go", "test", "./"+row['relative_go_package'], '-run', '/'.join(test_fns), '-json', '-coverprofile', cover_file_path], cwd=project_path,
                        stdout=subprocess.PIPE, 
                        stderr=subprocess.PIPE,
    )
    #print("debug go test", p.returncode, p.stdout.decode("utf-8"), p.stderr.decode("utf-8"), "", sep="\n")
    all_passed = True
    if p.returncode != 0:
        all_passed = False
        test_stderr = p.stderr
        #print('code', p.returncode, 'strderr', p.stderr.decode("utf-8"), 'stdout', p.stdout.decode("utf-8"))
    passed = 0
    failed = 0
    root_passed = 0
    root_failed = 0
    for line in p.stdout.decode('utf-8').split('\n'):
        if line == "":
            continue
        line_data = json.loads(line)
        if line_data.get('Action') == 'build-fail':
            raise Exception(f"go test build failed: stdout = {p.stdout}, stderr = {p.stderr}")
        if 'Test' not in line_data:
            continue
        if 'Action' not in line_data:
            continue
        action = line_data['Action']
        if action == 'pass':
            if '/' in line_data['Test']:
                passed += 1
            else:
                root_passed += 1
        elif action == 'fail':
            if '/' in line_data['Test']:
                failed += 1
            else:
                root_failed += 1
        else:
            continue
            #print(json.loads(line))
            #data.append(json.loads(line))
        #print(f"cd {project_path}")
        #print(' '.join(["go", "test", "./"+row['relative_go_package'], '-run', '/'.join(test_fns)]))

    f = open(cover_file_path)
    coverout = f.read()
    f.close()

    f = open(cover_file_path, 'w')
    f.writelines([line+'\n' for line in coverout.split('\n') if row['relative_project_path'] in line or line.startswith('mode:')])
    f.close()

    p = subprocess.run(["go", "tool", "cover", "-func", cover_file_path], cwd=project_path,
                        stdout=subprocess.PIPE, 
                        stderr=subprocess.PIPE,
    )
    if p.returncode != 0:
        print('go tool cover: code', p.returncode, 'strderr', p.stderr)

    coverage = 0.0
    foundCoverage = False
    for line in p.stdout.decode('utf-8').split('\n'):
        if line.startswith('total:'):
            out = re.search(r"\(statements\)\W+(\d+\.?\d*)\%", line)
            coverage = float(out.group(1))
            foundCoverage = True
    if not foundCoverage:
        print('no total coverage, stdout:', p.stdout, 'stderr:', p.stderr)

    # clean
    os.remove(test_file_path)
    os.remove(cover_file_path)

    return {'root_passed': root_passed, 'root_failed': root_failed, "passed": passed, "failed": failed, "all_passed": all_passed, "coverage": coverage, 'test_stderr': test_stderr}

In [8]:
#cache = {}

def list_dict_column(items, k, default=0):
    return [item.get(k, default) if isinstance(item, dict) else default for item in items]

def process_file(rows) -> dict:
    size = len(rows['project_path'])
    main_file_contents = ['']*size
    read_time = [0.0]*size
    test_file_contents = ['']*size
    generate_time = [0.0]*size
    check_results = [None]*size
    check_result_time = [0.0]*size
    error_types = ['']*size
    errors = ['']*size

    for i in range(size):
        start = time.time()
        main_file_path = REPOS_DIR+'/'+rows['project_path'][i]+rows['relative_project_path'][i]
        try:
            with open(main_file_path) as f:
                main_file_contents[i] = f.read()
        except Exception as e:
            errors[i] = str(e)
            error_types[i] = 'read_main_file'
        read_time[i] = float(time.time() - start)

    start = time.time()
    content_size = 0
    for content in main_file_contents:
        content_size += len(content)
    try:
        with torch.no_grad():
            test_file_contents = generate_unit_test(main_file_contents)
        #if row['file_hash'] in cache:
        #    test_file_contents[i] = cache[row['file_hash']]
        #else:
            #test_file_contents[i] = generate_unit_test(main_file_content)
            #cache[row['file_hash']] = test_file_content
    except Exception as e:
        err = str(e)
        error_type = 'generate'
        if err.startswith('CUDA out of memory'):
            error_type = 'generate_oom'
            gc.collect()
            torch.cuda.empty_cache()
        err = f"[content size: {content_size}] "+str(e)
        for i in range(size):
            if errors[i] != '':
                continue
            errors[i] = err
            error_types[i] = error_type
    generate_time = [float(time.time() - start)]*size

    for i in range(size):
        if errors[i] != '':
            continue
        start = time.time()
        try:
            row = {key: values[i] for key, values in rows.items()}
            check_results[i] = check_result(row, test_file_contents[i])
        except Exception as e:
            errors[i] = str(e)
            error_types[i] = 'other'
            if errors[i].startswith('go mod tidy: '):
                error_types[i] = 'deps'
            if errors[i].startswith('goimports: '):
                error_types[i] = 'goimports'
            if errors[i].startswith('go test build failed: '):
                error_types[i] = 'test_build_failed'            
        check_result_time[i] = float(time.time() - start)

    print('content_size', content_size)
    print('error_types', error_types)
    print('errors', errors)

    return {
        'root_passed': list_dict_column(check_results, 'root_passed'),
        'root_failed': list_dict_column(check_results, 'root_failed'),
        'passed': list_dict_column(check_results, 'passed'),
        'failed': list_dict_column(check_results, 'failed'),
        'all_passed': list_dict_column(check_results, 'all_passed', False),
        'coverage': list_dict_column(check_results, 'coverage', 0.0),
        'test_stderr': list_dict_column(check_results, 'test_stderr', b''),
        'test_file_content': test_file_contents,
        'error_type': error_types,
        'error': errors,
        'read_time': read_time,
        'generate_time': generate_time,
        'check_result_time': check_result_time,
    }

In [ ]:
row = {'path': '/pkg/markdown/convert.go',
 'repo_name': 'avelino/awesome-go',
 'revision_id': 'c3643eb9da5c673101f8fe15a6deb40bfc4a1c85',
 'star_events_count': 112752,
 'fork_events_count': 13739,
 'is_vendor': False,
 'is_generated': False,
 'length_bytes': 1342,
 'extension': 'go',
 'filename': 'convert.go',
 'file_exists': True,
 'file_hash': 'f0035e6fd1be46605da254d27a285c1ec41f081895fc73376b18219132a5f04d',
 'project': '/',
 'relative_project_path': 'pkg/markdown/convert.go',
 'relative_go_package': 'pkg/markdown/',
 'project_path': 'c3643eb9da5c673101f8fe15a6deb40bfc4a1c85/'}

row = ds.filter(lambda row: row['repo_name'] == 'avelino/awesome-go' and row['path'] == '/pkg/slug/generator.go')[0]
print(row)
#row = ds.filter(lambda row: row['repo_name'] == 'flipped-aurora/gin-vue-admin' and row['path'] == '/server/utils/ast/ast_auto_enter.go')[0]
#print(row)
# row = ds.filter(lambda row: row['repo_name'] == 'flipped-aurora/gin-vue-admin' and row['path'] == '/server/model/system/sys_dictionary_detail.go')[0]
# print(row)
row = ds.filter(lambda row: row['repo_name'] == 'dapr/dapr' and row['path'] == '/pkg/actors/actor_lock.go')[0]
print(row)

res = process_file(row)

print(res)
input_file = f"{REPOS_DIR}/{row['project_path']}{row['relative_project_path']}"
print(f"## input {row['repo_name']} {row['project_path']} {row['relative_go_package']} {input_file}")
with open(input_file) as f:
    print(f.read())
print('## output')
print(res['test_file_content'])
    

In [9]:
gc.collect()
torch.cuda.empty_cache()

res_ds2 = ds.shuffle().take(20).map(process_file, num_proc=1, batched=True, batch_size=1)

# single batch = 1600 - 483.5min

#res_ds.save_to_disk('data/res_ds')

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

content_size 2219
error_types ['']
errors ['']
content_size 8015
error_types ['test_build_failed']
errors ['go test build failed: stdout = b\'{"ImportPath":"code.gitea.io/gitea/services/packages/cargo [code.gitea.io/gitea/services/packages/cargo.test]","Action":"build-output","Output":"# code.gitea.io/gitea/services/packages/cargo [code.gitea.io/gitea/services/packages/cargo.test]\\\\n"}\\n{"ImportPath":"code.gitea.io/gitea/services/packages/cargo [code.gitea.io/gitea/services/packages/cargo.test]","Action":"build-output","Output":"services/packages/cargo/index_llm_test.go:15:7: undefined: test.InitGitea\\\\n"}\\n{"ImportPath":"code.gitea.io/gitea/services/packages/cargo [code.gitea.io/gitea/services/packages/cargo.test]","Action":"build-output","Output":"services/packages/cargo/index_llm_test.go:15:25: undefined: setting.TestingMode\\\\n"}\\n{"ImportPath":"code.gitea.io/gitea/services/packages/cargo [code.gitea.io/gitea/services/packages/cargo.test]","Action":"build-output","Output":"

KeyboardInterrupt: 

In [62]:
gc.collect()
torch.cuda.empty_cache()

res_ds2 = ds.shuffle().take(20).map(process_file, num_proc=1, batched=True, batch_size=4)

# single batch = 1600 - 483.5min

#res_ds.save_to_disk('data/res_ds')

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

content_size 13087
error_types ['', 'test_build_failed', 'test_build_failed', 'test_build_failed']
errors ['', 'go test build failed: stdout = b\'{"ImportPath":"github.com/derailed/k9s/internal/xray [github.com/derailed/k9s/internal/xray.test]","Action":"build-output","Output":"# github.com/derailed/k9s/internal/xray [github.com/derailed/k9s/internal/xray.test]\\\\n"}\\n{"ImportPath":"github.com/derailed/k9s/internal/xray [github.com/derailed/k9s/internal/xray.test]","Action":"build-output","Output":"internal/xray/svc_llm_test.go:18:3: undefined: t\\\\n"}\\n{"ImportPath":"github.com/derailed/k9s/internal/xray [github.com/derailed/k9s/internal/xray.test]","Action":"build-fail"}\\n{"Time":"2025-05-01T02:16:32.660252251+03:00","Action":"start","Package":"github.com/derailed/k9s/internal/xray"}\\n{"Time":"2025-05-01T02:16:32.660286718+03:00","Action":"output","Package":"github.com/derailed/k9s/internal/xray","Output":"FAIL\\\\tgithub.com/derailed/k9s/internal/xray [build failed]\\\\n"}\\n{

In [ ]:
print(res_ds2.to_pandas()['read_time'].mean())
print(res_ds2.to_pandas()['generate_time'].mean())
print(res_ds2.to_pandas()['check_result_time'].mean())
res_ds2.to_pandas()[['', 'generate_time']]

0.4288469791412354
43.199986171722415
2.7197426199913024


0     56.745275
1     56.745275
2     56.745275
3     56.745275
4     60.254207
5     60.254207
6     60.254207
7     60.254207
8     14.659992
9     14.659992
10    14.659992
11    14.659992
12    81.901206
13    81.901206
14    81.901206
15    81.901206
16     2.439252
17     2.439252
18     2.439252
19     2.439252
Name: generate_time, dtype: float64

In [144]:
res_ds.save_to_disk('data/res_ds')

Saving the dataset (0/1 shards):   0%|          | 0/1600 [00:00<?, ? examples/s]

In [55]:
#print(res_ds.to_pandas().query('error != ""')[['path', 'revision_id', 'repo_name', 'relative_go_package', 'error']])

for _, row in res_ds2.to_pandas().query('error != ""').iterrows():
    print(row['repo_name'], f"{REPOS_DIR}/{row['project_path']}", row['relative_go_package'], row['path'], row['error'], row['test_file_content'], sep="\n")

go-gitea/gitea
/home/djvue/dev/vkr/data/repos/9a3de436f452144933888fb1e4f336efeb93293a/
routers/web/
/routers/web/webfinger.go
[content size: 53937] CUDA out of memory. Tried to allocate 1.31 GiB. GPU 0 has a total capacity of 11.60 GiB of which 1.13 GiB is free. Including non-PyTorch memory, this process has 9.58 GiB memory in use. Of the allocated memory 6.90 GiB is allocated by PyTorch, and 2.46 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

go-gitea/gitea
/home/djvue/dev/vkr/data/repos/9a3de436f452144933888fb1e4f336efeb93293a/
routers/web/repo/
/routers/web/repo/view.go
[content size: 53937] CUDA out of memory. Tried to allocate 1.31 GiB. GPU 0 has a total capacity of 11.60 GiB of which 1.13 GiB is free. Including non-PyTorch memory, this proces

In [147]:
import numpy as np

# res_ds = datasets.load_from_disk('data/res_ds')

# print(res_ds.to_pandas()['passed'].value_counts())

# print(res_ds.to_pandas()['failed'].value_counts())

# print(res_ds.to_pandas()['all_passed'].value_counts())

# print(np.average(res_ds.to_pandas()['coverage']))

# print(len(res_ds.to_pandas().query('error != ""')))

total_files = len(res_ds)
success_files = len(res_ds.to_pandas().query('error == ""'))
error_files = len(res_ds.to_pandas().query('error != ""'))
all_passed_files = len(res_ds.to_pandas().query('all_passed'))
root_passed_tests = res_ds.to_pandas()['root_passed'].sum()
root_failed_tests = res_ds.to_pandas()['root_failed'].sum()
total_root_tests = root_passed_tests+root_failed_tests
passed_tests = res_ds.to_pandas()['passed'].sum()
failed_tests = res_ds.to_pandas()['failed'].sum()
total_tests = passed_tests+failed_tests
average_coverage = res_ds.to_pandas().query('error == ""')['coverage'].mean()

pd.DataFrame(data=[
    ['total_files', total_files, 1],
    ['success_files', success_files, success_files/total_files],
    ['error_files', error_files, error_files/total_files],
    ['all_passed_files', all_passed_files, all_passed_files/total_files],
    ['total_root_tests', total_root_tests, 1],
    ['root_passed_tests', root_passed_tests, root_passed_tests/total_root_tests],
    ['root_failed_tests', root_failed_tests, root_failed_tests/total_root_tests],
    ['total_tests', total_tests, 1],
    ['passed_tests', passed_tests, passed_tests/total_tests],
    ['failed_tests', failed_tests, failed_tests/total_tests],
    ['average_coverage', average_coverage, average_coverage],
], columns=['name', 'count', 'part']).set_index('name')

,count,part
name,,
total_files,1600.000000,1.000000
success_files,452.000000,0.282500
error_files,1148.000000,0.717500
all_passed_files,183.000000,0.114375
total_root_tests,475.000000,1.000000
root_passed_tests,214.000000,0.450526
root_failed_tests,261.000000,0.549474
total_tests,408.000000,1.000000
passed_tests,204.000000,0.500000


In [148]:
res_ds.to_pandas()['error_type'].value_counts()

error_type
test_build_failed    865
                     452
goimports            202
deps                  80
other                  1
Name: count, dtype: int64

In [72]:
print(len(res_ds.to_pandas().query('error == "" and root_passed == 0 and root_failed == 0 and passed == 0 and failed == 0')))
for _, row in res_ds.to_pandas().query('error == "" and root_passed == 0 and root_failed == 0 and passed == 0 and failed == 0').head(10).iterrows():
    input_file = f"{REPOS_DIR}/{row['project_path']}{row['relative_project_path']}"
    print(f"## input {row['repo_name']} {row['project_path']} {input_file}")
    with open(input_file) as f:
        print(f.read())
    print('## output')
    print(row['test_file_content'])

1
## input etcd-io/etcd 2888a95d5d4f20e7e852214e888d79ead642300b/etcdctl/ /home/djvue/dev/vkr/data/repos/2888a95d5d4f20e7e852214e888d79ead642300b/etcdctl/ctlv3/command/get_command.go
// Copyright 2015 The etcd Authors
//
// Licensed under the Apache License, Version 2.0 (the "License");
// you may not use this file except in compliance with the License.
// You may obtain a copy of the License at
//
//     http://www.apache.org/licenses/LICENSE-2.0
//
// Unless required by applicable law or agreed to in writing, software
// distributed under the License is distributed on an "AS IS" BASIS,
// WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
// See the License for the specific language governing permissions and
// limitations under the License.

package command

import (
	"fmt"
	"strings"

	"github.com/spf13/cobra"

	clientv3 "go.etcd.io/etcd/client/v3"
	"go.etcd.io/etcd/pkg/v3/cobrautl"
)

var (
	getConsistency string
	getLimit       int64
	getSortOrder   string
	

In [76]:
#system_message = "The user is going to give you code and would like to have tests written in the Python unittest module."
# see https://developers.cloudflare.com/workers-ai/guides/tutorials/explore-code-generation-using-deepseek-coder-models/
# see docs deepseek-coder v2 https://github.com/deepseek-ai/DeepSeek-Coder-V2
# see https://github.com/deepseek-ai/DeepSeek-Coder
# see https://github.com/raiharsh88/tgen/blob/main/llama_server/generator/llama.py

# You are an expert programmer. 
# The user is going to give you code and would like to have tests
# Based on the following instructions and given code files, write unit tests for the first file;
# all the other files are just dependencies to give you context of all the possible test cases to produce.

# Instructions:
# 1. Each file wrapped between [File] and [/File] tags.
# 2. You should only return output test file.
# 3. Cover all possible inputs and their respective outputs using tests.
system_message = """
You are an expert programmer. 
You should only return output test file containing working code.
The user is going to give you code and would like to have unit tests for the first file.
All the other files are just dependencies to give you context of all the possible test cases to produce.
Cover all possible inputs and their respective outputs using tests.
"""

code = """
```python
#user.py
from password import Password

class User:

    def __init__(self, first_name, last_name=None, password: Password=None):
        self.first_name = first_name
        self.last_name = last_name
        self.password = password
        if last_name is None:
            self.last_name = "Mc" + self.first_name

    def full_name(self):
        return self.first_name + " " + self.last_name + " " + self.password.get_password()
```

```python
#password.py
class Password:

    def __init__(self, password):
        self._password = password

    def get_password():
        return self._password.lower()
```
"""

messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "content": code},
]
inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
# tokenizer.eos_token_id is the id of <|EOT|> token
model.generation_config.pad_token_id = tokenizer.pad_token_id
outputs = model.generate(inputs, max_new_tokens=512, do_sample=False, top_k=50,
    # top_p=0.95,
    num_return_sequences=1,
    pad_token_id=tokenizer.eos_token_id,eos_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True))

Here is a set of unit tests for the `user.py` file.

```python
import unittest
from user import User
from password import Password

class TestUser(unittest.TestCase):

    def setUp(self):
        self.password = Password("TestPassword123")
        self.user = User("John", "Doe", self.password)

    def test_full_name(self):
        self.assertEqual(self.user.full_name(), "John Doe TestPassword123")

    def test_full_name_with_no_last_name(self):
        user = User("Jane")
        self.assertEqual(user.full_name(), "Jane McJane")

    def test_full_name_with_no_password(self):
        user = User("Jane", None)
        self.assertEqual(user.full_name(), "Jane McJane")

if __name__ == '__main__':
    unittest.main()
```

In this test suite, we are testing the `full_name` method of the `User` class. We are also testing the `full_name` method of the `User` class with no last name and no password.

The `setUp` method is used to create an instance of the `User` class and an instance of the

In [35]:
input_text = "#write a quick sort algorithm"
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_length=1024)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


#write a quick sort algorithm in python


def quick_sort(arr):
    if len(arr) <= 1:
        return arr
    else:
        pivot = arr[0]
        less_than_pivot = [x for x in arr[1:] if x <= pivot]
        greater_than_pivot = [x for x in arr[1:] if x > pivot]
        return quick_sort(less_than_pivot) + [pivot] + quick_sort(greater_than_pivot)

print(quick_sort([3,6,8,10,1,2,1]))
#output: [1, 1, 2, 3, 6, 8, 10]

#This algorithm works by selecting a 'pivot' element from the array and partitioning the other elements into two sub-arrays, according to whether they are less than or greater than the pivot. The sub-arrays are then recursively sorted.

#The time complexity of the Quick Sort algorithm is O(n log n) in all 3 cases (worst, average and best) when the pivot is the smallest or the largest element.
#The space complexity is O(log n) in the best case and O(n) in the worst case.
#The worst case happens when the input array is already sorted.
#The best case happens when the pivot is the

In [39]:
input_text = """#utils.py
import torch
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

def load_data():
    iris = datasets.load_iris()
    X = iris.data
    y = iris.target

    # Standardize the data
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Convert numpy data to PyTorch tensors
    X_train = torch.tensor(X_train, dtype=torch.float32)
    X_test = torch.tensor(X_test, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.int64)
    y_test = torch.tensor(y_test, dtype=torch.int64)
    
    return X_train, X_test, y_train, y_test

def evaluate_predictions(y_test, y_pred):
    return accuracy_score(y_test, y_pred)
#model.py
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

class IrisClassifier(nn.Module):
    def __init__(self):
        super(IrisClassifier, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(4, 16),
            nn.ReLU(),
            nn.Linear(16, 3)
        )

    def forward(self, x):
        return self.fc(x)

    def train_model(self, X_train, y_train, epochs, lr, batch_size):
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.parameters(), lr=lr)
        
        # Create DataLoader for batches
        dataset = TensorDataset(X_train, y_train)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

        for epoch in range(epochs):
            for batch_X, batch_y in dataloader:
                optimizer.zero_grad()
                outputs = self(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()

    def predict(self, X_test):
        with torch.no_grad():
            outputs = self(X_test)
            _, predicted = outputs.max(1)
        return predicted.numpy()
#main.py
from utils import load_data, evaluate_predictions
from model import IrisClassifier as Classifier

def main():
    # Model training and evaluation
"""
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=140)
print(tokenizer.decode(outputs[0]))

Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


<｜begin▁of▁sentence｜>#utils.py
import torch
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

def load_data():
    iris = datasets.load_iris()
    X = iris.data
    y = iris.target

    # Standardize the data
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Convert numpy data to PyTorch tensors
    X_train = torch.tensor(X_train, dtype=torch.float32)
    X_test = torch.tensor(X_test, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.int64)
    y_test = torch.tensor(y_test, dtype=torch.int64)

    return X_train, X_test, y_train, y_test

def evaluate_predictions(y_test, y_pred):
    return accuracy_score(y_test, y_pred)
#model.py
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, 

In [4]:
ds_res_1600 = datasets.load_from_disk('./data/res_ds_1600')

for row in ds_res_1600.filter(lambda x: x['relative_project_path'] == 'pkg/markdown/convert.go'):
    print("('"+row["project_path"]+"', '"+row['relative_go_package']+"', '"+row['relative_project_path']+"', \"\"\"\n"+row['test_file_content']+"\n\"\"\")")

for row in ds_res_1600.take(1000).shuffle().take(10):
    print("('"+row["project_path"]+"', '"+row['relative_go_package']+"', '"+row['relative_project_path']+"', \"\"\"\n"+row['test_file_content']+"\n\"\"\")")

Filter:   0%|          | 0/1600 [00:00<?, ? examples/s]

('c3643eb9da5c673101f8fe15a6deb40bfc4a1c85/', 'pkg/markdown/', 'pkg/markdown/convert.go', """

package markdown

import (
	"testing"
)

func TestToHTML(t *testing.T) {
	testCases := []struct {
		name     string
		input    string
		expected string
	}{
		{
			name:     "simple markdown",
			input:    "# Hello World\n\nThis is some markdown.",
			expected: "<h1>Hello World</h1>\n\n<p>This is some markdown.</p>\n",
		},
		{
			name:     "complex markdown",
			input:    "# Hello World\n\nThis is a [link](https://example.com).\n\nAnd another one.",
			expected: "<h1>Hello World</h1>\n\n<p>This is a <a href=\"https://example.com.\">link</a>.</p>\n\n<p>And another one.</p>\n",
		},
		// Add more test cases as needed
	}

	for _, tc := range testCases {
		t.Run(tc.name, func(t *testing.T) {
			html, err := ToHTML([]byte(tc.input))
			if err != nil {
				t.Errorf("ToHTML failed: %v", err)
			}

			if string(html) != tc.expected {
				t.Errorf("Expected %s, got %s", tc.expected, string(html))
			}